In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.models import Model
from sklearn.metrics import classification_report, confusion_matrix


In [2]:
# Define directories
train_dir = 'train'
test_dir = 'test'

# Define image dimensions and batch size
img_height, img_width = 150, 150
batch_size = 32

# Create data generators
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')


Found 2000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [3]:
# Load pre-trained VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

# Freeze convolutional layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers for SVM classifier
x = Flatten()(base_model.output)
predictions = Dense(1, activation='sigmoid')(x)

# Create SVM model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=5,
    validation_data=test_generator,
    validation_steps=test_generator.samples // batch_size)


Epoch 1/5


C:\Users\sayak\Desktop\Essentials\Internship\venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


62/62 ━━━━━━━━━━━━━━━━━━━━ 220s 4s/step - accuracy: 0.7822 - loss: 0.4649 - val_accuracy: 0.8236 - val_loss: 0.3706
Epoch 2/5
 1/62 ━━━━━━━━━━━━━━━━━━━━ 1:47 2s/step - accuracy: 0.8125 - loss: 0.4703

C:\Users\sayak\AppData\Local\Programs\Python\Python312\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


62/62 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.8125 - loss: 0.2390 - val_accuracy: 0.8750 - val_loss: 0.1521
Epoch 3/5
62/62 ━━━━━━━━━━━━━━━━━━━━ 219s 4s/step - accuracy: 0.8994 - loss: 0.2566 - val_accuracy: 0.8896 - val_loss: 0.2674
Epoch 4/5
62/62 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.9062 - loss: 0.1085 - val_accuracy: 0.9375 - val_loss: 0.0880
Epoch 5/5
62/62 ━━━━━━━━━━━━━━━━━━━━ 220s 4s/step - accuracy: 0.9242 - loss: 0.2066 - val_accuracy: 0.8916 - val_loss: 0.2559


In [4]:
# Evaluate the model
test_loss, test_acc = model.evaluate(test_generator, verbose=2)
print(f'Test accuracy: {test_acc}')

# Predictions
predictions = model.predict(test_generator)
y_pred = np.where(predictions > 0.5, 1, 0)

# Print classification report and confusion matrix
print(classification_report(test_generator.classes, y_pred))
print(confusion_matrix(test_generator.classes, y_pred))


63/63 - 110s - 2s/step - accuracy: 0.8920 - loss: 0.2543
Test accuracy: 0.8920000195503235
63/63 ━━━━━━━━━━━━━━━━━━━━ 111s 2s/step
              precision    recall  f1-score   support

           0       0.51      0.52      0.52      1000
           1       0.51      0.49      0.50      1000

    accuracy                           0.51      2000
   macro avg       0.51      0.51      0.51      2000
weighted avg       0.51      0.51      0.51      2000

[[524 476]
 [506 494]]
